In [1]:
import os
import scanpy as sc
import numpy as np
import multiprocessing
import anndata
import pandas as pd

from sklearn.cluster import KMeans
from scipy import sparse


/Users/antonogorodnikov/anaconda3/envs/scito/lib/python3.8/site-packages/anndata/core/anndata.py:17: FutureWarning: pandas.core.index is deprecated and will be removed in a future version.  The public classes are available in the top-level namespace.
  from pandas.core.index import RangeIndex


In [2]:
path = '/Users/antonogorodnikov/Documents/Work/DataSci/SCITO-seq/tests/100k_pbmc_filtered_feature_bc_matrix.h5'

In [3]:
batchid_string='barcode'

In [4]:
adata = sc.read_10x_h5(path, gex_only=False)

Variable names are not unique. To make them unique, call `.var_names_make_unique`.


In [5]:
n_clust=None
batches = adata.var_names.str.extract(r'(%s\d+)'%batchid_string).iloc[:,0].dropna().unique()
nClust = n_clust if n_clust != None else len(batches)+1

In [6]:
ab_adata = adata[:,adata.var_names.str.contains(r'(%s\d+)'%batchid_string)]

/Users/antonogorodnikov/anaconda3/envs/scito/lib/python3.8/site-packages/pandas/core/strings.py:1952: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


In [7]:
def count_collapser(data, bc):
    '''
    Collapses counts for all antibodies for a given batch barcode. Outputs dense data structure
    :param data: sparse or dense data (AnnData or DataFrame) with counts
    :param bc: batch barcode
    :return: Numpy array of cells by batch BC.
    '''
    #TODO add handle for dense data
    rel_data = data[:,data.var.index.str.contains(r'(%s$)'%bc)]
    collapsed = rel_data.X.toarray().sum(axis=1).astype("int")

    return collapsed

In [8]:
batch_counts = np.transpose(np.array([count_collapser(ab_adata, bc) for bc in batches]))

In [9]:
# create anndata object with collapsed counts per batch
batch_adata = anndata.AnnData(X=sparse.csr_matrix(batch_counts),
                              obs=adata.obs,
                              var=pd.DataFrame(batches, columns=['batch']))

Transforming to str index.


In [10]:
sc.pp.normalize_per_cell(batch_adata, counts_per_cell_after=1e4)
#sc.pp.log1p(batch_adata)
batch_adata

AnnData object with n_obs × n_vars = 35371 × 10 
    obs: 'n_counts'
    var: 'batch'

In [11]:
n_clust=None
n_init=100
kfunc="kmeans"
maxneighbor=100
seed=42
keep_input=False

In [12]:
clust_model = KMeans(n_clusters=nClust, n_init=n_init, random_state=seed, n_jobs=int(multiprocessing.cpu_count()*0.6))
clusters = clust_model.fit_predict(batch_counts)


In [13]:
batch_adata

AnnData object with n_obs × n_vars = 35371 × 10 
    obs: 'n_counts'
    var: 'batch'

In [14]:
# Dont change
def av_gene_expression(anndata, marker_dict, gene_symbol_key=None, partition_key='batch_cluster'):
    """ Copied from https://github.com/theislab/scanpy/issues/181 - posted by one of scanpy developers
    A function go get mean z-score expressions of feature per cluster (class)
    #
    # Inputs:
    #    anndata         - An AnnData object containing the data set and a partition
    #    marker_dict     - A dictionary with cell-type markers. The markers should be stores as anndata.var_names or
    #                      an anndata.var field with the key given by the gene_symbol_key input
    #    gene_symbol_key - The key for the anndata.var field with gene IDs or names that correspond to the marker
    #                      genes
    #    partition_key   - The key for the anndata.obs field where the cluster IDs are stored. The default is
    #                      'batch_cluster' """

    # Test inputs
    if partition_key not in anndata.obs.columns.values:
        print('KeyError: The partition key was not found in the passed AnnData object.')
        print('   Have you done the clustering? If so, please tell pass the cluster IDs with the AnnData object!')
        raise

    if (gene_symbol_key != None) and (gene_symbol_key not in anndata.var.columns.values):
        print('KeyError: The provided gene symbol key was not found in the passed AnnData object.')
        print('   Check that your cell type markers are given in a format that your anndata object knows!')
        raise

    if gene_symbol_key:
        gene_ids = anndata.var[gene_symbol_key]
    else:
        gene_ids = anndata.var_names

    clusters = set(anndata.obs[partition_key])
    n_clust = len(clusters)
    marker_exp = pd.DataFrame(columns=clusters)
    marker_exp['cell_type'] = pd.Series({}, dtype='str')
    marker_names = []

    z_scores = sc.pp.scale(anndata, copy=True, zero_center=False)

    i = 0
    for group in marker_dict:
        # Find the corresponding columns and get their mean expression in the cluster
        for gene in marker_dict[group]:
            ens_idx = np.in1d(gene_ids, gene)  # Note there may be multiple mappings
            if np.sum(ens_idx) == 0:
                continue
            else:
                z_scores.obs[ens_idx[0]] = z_scores.X[:, ens_idx].mean(1)  # works for both single and multiple mapping
                ens_idx = ens_idx[0]

            clust_marker_exp = z_scores.obs.groupby(partition_key)[ens_idx].apply(np.mean).tolist()
            clust_marker_exp.append(group)
            marker_exp.loc[i] = clust_marker_exp
            marker_names.append(gene)
            i += 1

    # Replace the rownames with informative gene symbols
    marker_exp.index = marker_names

    return (marker_exp)


In [15]:
marker_dict={batch_adata.var.keys()[0]: batch_adata.var['batch']}

In [16]:
batch_adata.obs['batch_cluster'] = clusters

In [17]:
av_gene_expression(batch_adata, marker_dict, gene_symbol_key='batch', partition_key='batch_cluster')

,0,1,2,3,4,5,6,7,8,9,10,cell_type
barcode1,0.203231,4.206511,0.001272,0.292635,0.023208,4.072051,0.193131,2.793429,0.199530,0.197400,0.007192,batch
barcode2,3.068106,0.000639,0.001102,0.253333,0.004399,0.002208,0.375388,0.177102,0.174944,0.185789,0.005443,batch
barcode3,0.168932,0.000704,0.000956,0.252115,0.003941,0.049483,2.923872,0.170008,0.183007,0.179390,0.004519,batch
barcode4,0.218435,0.000653,0.001202,0.750875,0.029274,0.001206,0.202652,0.213478,0.220353,0.205166,0.005298,batch
barcode5,0.219544,0.004051,0.020069,0.808039,0.005113,0.002092,0.208328,0.223306,0.240596,0.205833,6.098987,batch
barcode6,0.195221,0.000616,0.010317,0.457736,0.089038,0.001724,0.180255,0.188916,0.196361,3.365077,0.040898,batch
barcode7,0.230624,0.000906,0.000990,0.785763,0.005355,0.001733,0.213460,0.227272,0.233675,0.218348,0.005888,batch
barcode8,0.166754,0.000909,0.064687,0.246165,0.040873,0.001596,0.152295,0.160836,3.187574,0.162449,0.005275,batch
barcode9,0.226492,0.000938,5.450461,0.754480,5.334151,0.053113,0.199177,0.227773,0.227157,0.207690,0.005566,batch
barcode10,0.194421,0.008415,0.000986,0.706203,0.002183,0.068181,0.172467,0.192769,0.212241,0.187663,0.002682,batch


In [1]:
np.transpose(batch_adata.X.todense())[:10,:10]

NameError: name 'np' is not defined

In [20]:
batch_adata.var.keys()[0]

'batch'